In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import fbprophet
import datetime

# ****Reading the dataset****

In [ ]:
df = pd.read_csv('../input/ethereumpricetill24072021/ethereum_data.csv')
df.head()

In [ ]:
df = df.rename(columns = {"Unnamed: 0":"Date"})
df.head()

In [ ]:
data = [df['Date'],df['close']]
headers = ['ds','y']
df_ts = pd.concat(data,axis = 1,keys=headers)
df_ts.sort_values(by='ds',ascending = True)
df_ts.head()



In [ ]:
plt.figure(figsize = (10,6))
plt.plot(df_ts['ds'],df_ts['y'])
plt.show()

In [ ]:
df_ts['ds'] = pd.to_datetime(df_ts['ds'])
df_ts.info()

**Creating Model Instance**

In [ ]:
model = fbprophet.Prophet()
model.fit(df_ts)

# Making an Insample Forecast

In [ ]:
past_dates = []
dates = list(pd.date_range('2021-06-21',periods=20).strftime('%Y-%m-%d'))
past_dates.append(dates)
past_dates

In [ ]:
past = pd.DataFrame(past_dates[0],columns = ['ds'])
past['ds'] = pd.to_datetime(past['ds'])
past.head()

In [ ]:
past_forecast = model.predict(past)
print(type(past_forecast))

In [ ]:
forecasted_past = past_forecast[['ds','yhat','yhat_lower','yhat_upper']]
forecasted_past

**Plotting to show the relationship between Predicted  and Actual In Insample Data**

In [ ]:
pred_dates= []
k = forecasted_past['ds'].values
for items in k:
    x = str(pd.to_datetime(items).date())
    pred_dates.append(x)

In [ ]:
up = []
low = []
adj = []
for dates in pred_dates:
    upper_limit = df[df['Date']==dates]['high'].reset_index(drop= True)
    lower_limit = df[df['Date']==dates]['low'].reset_index(drop= True)
    adjust = df[df['Date']==dates]['close'].reset_index(drop= True)
    up.append(upper_limit[0])
    low.append(lower_limit[0])
    adj.append(adjust[0])

In [ ]:
plt.figure(figsize = (10,6))
plt.plot(forecasted_past['ds'],forecasted_past['yhat_upper'],'-g')
plt.plot(forecasted_past['ds'],up,'-b')
plt.legend(['Predicted','True'])
plt.title('Upper Ranges')
plt.show()


In [ ]:
plt.figure(figsize = (10,6))
plt.plot(forecasted_past['ds'],forecasted_past['yhat_lower'],'-r')
plt.plot(forecasted_past['ds'],low,'-b')
plt.legend(['Predicted','True'])
plt.title('Lower Ranges')
plt.show()

In [ ]:
plt.figure(figsize = (10,6))
plt.plot(forecasted_past['ds'],forecasted_past['yhat'],'-b')
plt.plot(forecasted_past['ds'],adj,'-g')
plt.legend(['Predicted','True'])
plt.title('Adjusted Range')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error
mse = mean_absolute_error(df_ts['y'].iloc[-len(forecasted_past['yhat']):],forecasted_past['yhat'])
print(mse)

In [ ]:
plt.figure(figsize = (10,6))
model.plot(past_forecast)
plt.show()

In [ ]:
plt.figure(figsize = (10,6))
model.plot_components(past_forecast)
plt.show()

# Making an outsample forecast

In [ ]:
import datetime
last_date_on_data = datetime.datetime.strptime(df['Date'].iloc[-1],'%Y-%m-%d')
todays_date = datetime.datetime.now().today()
days = (todays_date-last_date_on_data).days

In [ ]:
future_dates = []
dates = list(pd.date_range(last_date_on_data,periods=days+20).strftime('%Y-%m-%d'))
future_dates.append(dates)

In [ ]:
future_df = pd.DataFrame(future_dates[0],columns = ['ds'])
future_df

In [ ]:
future_forecast = model.predict(future_df)
# print(type(future_forecast))

In [ ]:
plt.figure(figsize = (10,6))
model.plot(future_forecast)
plt.show()

In [ ]:
plt.figure(figsize = (10,6))
model.plot_components(future_forecast)
plt.show()

In [ ]:
plt.figure(figsize = (10,6))
plt.plot(future_forecast['ds'],future_forecast['yhat_lower'],'-r')
plt.plot(future_forecast['ds'],future_forecast['yhat'],'-b')
plt.plot(future_forecast['ds'],future_forecast['yhat_upper'],'-g')
plt.legend(['Day Lowest','Adjusted Closing Price','Day Highest'],loc = 'best')
plt.show()